In [16]:
import time
from pathlib import Path
from typing import Optional
from reolink_api_client import ReolinkAPIClient
import requests


In [17]:
# ===========================
# CONFIGURATION
# ===========================
PI_IP = "192.168.255.14"
API_BASE = f"http://{PI_IP}:8081"


INITIAL_PRESET = 20
FIRST_SAVE_POSE = 30
STEP_DEGREES = 20
TOTAL_DEGREES = 360*2
DIRECTION = "Right"   # "Left" or "Right"
SPEED_LEVEL = 3       # valid 1..5
SLEEP_AFTER_MOVE = 5  # seconds between moves
IMAGE_WIDTH = 1280

In [18]:
# ===========================
# INITIALIZE CLIENT
# ===========================
client = ReolinkAPIClient(API_BASE)

# Get list of cameras dynamically
try:
    camera_data = client.list_cameras()
    camera_ips = camera_data.get("camera_ips", [])
    print(f"Found {len(camera_ips)} camera(s): {camera_ips}")
except Exception as e:
    raise RuntimeError(f"Failed to list cameras: {e}")

Found 2 camera(s): ['192.168.1.11', '192.168.1.12']


In [19]:
# ===========================
# HELPERS
# ===========================
def set_preset(client: ReolinkAPIClient, camera_ip: str, idx: Optional[int] = None):
    params = {"camera_ip": camera_ip}
    if idx is not None:
        params["idx"] = str(idx)
    resp = requests.post(f"{client.base_url}/control/preset/set", params=params, timeout=15)
    resp.raise_for_status()
    return resp.json()


def ensure_dirs_for_camera(camera_ip: str):
    root_dir = Path(f"captures/{PI_IP}") / camera_ip
    out_dir = root_dir / "images"
    out_dir.mkdir(parents=True, exist_ok=True)
    return root_dir, out_dir


def capture_and_save(client: ReolinkAPIClient, camera_ip: str, out_dir: Path, pose_id: int, width: int = 1280):
    # capture_image now supports width parameter
    img = client.capture_image(camera_ip, width=width)  # returns PIL.Image
    p = out_dir / f"pose_{pose_id:02d}.jpg"
    img.save(p, quality=95)
    print(f"Saved {p} ({width}px wide)")

In [20]:

# ===========================
# RUN SWEEP FOR EACH CAMERA
# ===========================
steps = int(TOTAL_DEGREES // STEP_DEGREES)

for cam_ip in camera_ips:
    print("\n" + "=" * 40)
    print(f"Starting sweep for {cam_ip}")
    root_dir, out_dir = ensure_dirs_for_camera(cam_ip)

    try:
        print("Stopping patrol if running")
        try:
            client.stop_patrol(cam_ip)
            time.sleep(2)
        except Exception as e:
            print(f"Warning, could not stop patrol, {e}")

        print(f"Moving to preset {INITIAL_PRESET}")
        client.move_camera(cam_ip, pose_id=INITIAL_PRESET, speed=SPEED_LEVEL)
        time.sleep(2)

        current_pose = FIRST_SAVE_POSE
        print(f"Setting dynamic preset {current_pose}")
        try:
            set_preset(client, cam_ip, idx=current_pose)
        except Exception as e:
            print(f"Warning, set_preset failed at pose {current_pose}, {e}")

        print("Capturing initial frame")
        capture_and_save(client, cam_ip, out_dir, current_pose, width=IMAGE_WIDTH)

        for i in range(steps):
            print(f"Step {i + 1}/{steps} rotate {STEP_DEGREES} degrees {DIRECTION}")
            move_info = client.move_camera(cam_ip, direction=DIRECTION, degrees=STEP_DEGREES, speed=SPEED_LEVEL)
            print(move_info)
            #time.sleep(SLEEP_AFTER_MOVE)

            current_pose += 1
            print(f"Setting dynamic preset {current_pose}")
            try:
                set_preset(client, cam_ip, idx=current_pose)
            except Exception as e:
                print(f"Warning, set_preset failed at pose {current_pose}, {e}")

            capture_and_save(client, cam_ip, out_dir, current_pose, width=IMAGE_WIDTH)

        print(f"Sweep complete for {cam_ip}")

    except Exception as e:
        print(f"Error while sweeping {cam_ip}, {e}")
        continue


Starting sweep for 192.168.1.11
Stopping patrol if running
Moving to preset 20
Setting dynamic preset 30
Capturing initial frame
Saved captures/192.168.255.14/192.168.1.11/images/pose_30.jpg (1280px wide)
Step 1/36 rotate 20 degrees Right
{'status': 'ok', 'camera_ip': '192.168.1.11', 'direction': 'Right', 'degrees': 20.0, 'duration': 4.71, 'speed': 3, 'brand': 'reolink-823S2'}
Setting dynamic preset 31
Saved captures/192.168.255.14/192.168.1.11/images/pose_31.jpg (1280px wide)
Step 2/36 rotate 20 degrees Right
{'status': 'ok', 'camera_ip': '192.168.1.11', 'direction': 'Right', 'degrees': 20.0, 'duration': 4.71, 'speed': 3, 'brand': 'reolink-823S2'}
Setting dynamic preset 32
Saved captures/192.168.255.14/192.168.1.11/images/pose_32.jpg (1280px wide)
Step 3/36 rotate 20 degrees Right
{'status': 'ok', 'camera_ip': '192.168.1.11', 'direction': 'Right', 'degrees': 20.0, 'duration': 4.71, 'speed': 3, 'brand': 'reolink-823S2'}
Setting dynamic preset 33
Saved captures/192.168.255.14/192.168.1